# DataFrames Basics

## Prerrequisites

Install Spark and Java in VM

In [ ]:
# install Java8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# download spark 3.5.0
!wget -q https://apache.osuosl.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz

In [ ]:
ls -l # check the .tgz is there

total 782028
drwxr-xr-x 1 root root      4096 Jan 10 14:23 sample_data/
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz
-rw-r--r-- 1 root root 400395283 Sep  9 02:10 spark-3.5.0-bin-hadoop3.tgz.1


In [ ]:
# unzip it
!tar xf spark-3.5.0-bin-hadoop3.tgz

In [ ]:
!pip install -q findspark

In [ ]:

!pip install py4j

# For maps
!pip install folium
!pip install plotly

Define the environment

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "--master local[*] pyspark-shell"

Start Spark Session

---

In [ ]:
import findspark
findspark.init("spark-3.5.0-bin-hadoop3")# SPARK_HOME

from pyspark.sql import SparkSession

# create the session
spark = SparkSession \
        .builder \
        .appName("DataFrames Basics") \
        .master("local[*]") \
        .getOrCreate()

spark.version

'3.5.0'

In [ ]:
spark

In [ ]:
# For Pandas conversion optimization
spark.conf.set("spark.sql.execution.arrow.enabled", "true")

In [ ]:
# Import sql functions
from pyspark.sql.functions import *

Download datasets

In [ ]:
!mkdir -p dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/cars.json -P /dataset
!wget -q https://raw.githubusercontent.com/paponsro/spark_edem_2324/master/dataset/movies.json -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/bank.csv -P /dataset
!wget -q https://github.com/masfworld/datahack_docker/raw/master/zeppelin/data/vehicles.csv -P /dataset
!ls /dataset

bank.csv  cars.json  movies.json  vehicles.csv


In [ ]:
ls -l /dataset

total 1784
-rw-r--r-- 1 root root  461474 Jan 12 17:23 bank.csv
-rw-r--r-- 1 root root   74910 Jan 12 17:23 cars.json
-rw-r--r-- 1 root root 1274347 Jan 12 17:23 movies.json
-rw-r--r-- 1 root root    4370 Jan 12 17:23 vehicles.csv


## Examples

In [ ]:
from pyspark.sql.types import Row
from pyspark.sql.functions import *

In [ ]:
bankText = spark.sparkContext.textFile("/dataset/bank.csv")

#we have to: remove firt row (headers), map the rest, and create DF
bank = bankText.map(lambda lineaCsv: lineaCsv.split(";"))\
.filter(lambda s: s[0] != "\"age\"") \
.map(lambda row: Row(int(row[0]), row[1].replace("\"", ""), row[2].replace("\"", ""), row[3].replace("\"", ""), row[5].replace("\"", ""))) \
.toDF(["age", "job", "marital", "education", "balance"]) \
.withColumn("age", col("age").cast("int"))

bank.show(3)

+---+----------+-------+---------+-------+
|age|       job|marital|education|balance|
+---+----------+-------+---------+-------+
| 30|unemployed|married|  primary|   1787|
| 33|  services|married|secondary|   4789|
| 35|management| single| tertiary|   1350|
+---+----------+-------+---------+-------+
only showing top 3 rows



Read directly from JSON file to a DF

In [ ]:
carsDF = spark.read.option("inferSchema", True).json("/dataset/cars.json") # inferSchema requires one extra pass over the data

# if None is set, it uses de default value (default = False) you can also pass the schema manually

Read directly from csv

In [ ]:
bankDF = spark.read.option("header", "true").option("delimiter", ";").csv("/dataset/bank.csv")
bankDF.show(3)

+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
|age|       job|marital|education|default|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|  y|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
| 30|unemployed|married|  primary|     no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown| no|
| 33|  services|married|secondary|     no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure| no|
| 35|management| single| tertiary|     no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure| no|
+---+----------+-------+---------+-------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+---+
only showing top 3 rows



Showing a DF and print schema

In [ ]:
carsDF.show(2)
carsDF.printSchema()

+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|Acceleration|Cylinders|Displacement|Horsepower|Miles_per_Gallon|                Name|Origin|Weight_in_lbs|      Year|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
|        12.0|        8|       307.0|       130|            18.0|chevrolet chevell...|   USA|         3504|1970-01-01|
|        11.5|        8|       350.0|       165|            15.0|   buick skylark 320|   USA|         3693|1970-01-01|
+------------+---------+------------+----------+----------------+--------------------+------+-------------+----------+
only showing top 2 rows

root
 |-- Acceleration: double (nullable = true)
 |-- Cylinders: long (nullable = true)
 |-- Displacement: double (nullable = true)
 |-- Horsepower: long (nullable = true)
 |-- Miles_per_Gallon: double (nullable = true)
 |-- Name: string (nullable = true)
 |-- 

Get Rows

In [ ]:
carsDF.take(2)

[Row(Acceleration=12.0, Cylinders=8, Displacement=307.0, Horsepower=130, Miles_per_Gallon=18.0, Name='chevrolet chevelle malibu', Origin='USA', Weight_in_lbs=3504, Year='1970-01-01'),
 Row(Acceleration=11.5, Cylinders=8, Displacement=350.0, Horsepower=165, Miles_per_Gallon=15.0, Name='buick skylark 320', Origin='USA', Weight_in_lbs=3693, Year='1970-01-01')]

Count

In [ ]:
carsDF.count()

406

Schema

In [ ]:
# obtain a schema
carsSchema = carsDF.schema
print(type(carsSchema))
print(carsSchema)

<class 'pyspark.sql.types.StructType'>
StructType([StructField('Acceleration', DoubleType(), True), StructField('Cylinders', LongType(), True), StructField('Displacement', DoubleType(), True), StructField('Horsepower', LongType(), True), StructField('Miles_per_Gallon', DoubleType(), True), StructField('Name', StringType(), True), StructField('Origin', StringType(), True), StructField('Weight_in_lbs', LongType(), True), StructField('Year', StringType(), True)])


Custom Schemas

In [ ]:
example = spark.sparkContext.parallelize([("chevrolet chevelle malibu",18,"1970-01-01","USA"),
    ("buick skylark 320",15,"1970-01-01","USA")])

In [ ]:
exampleDF = spark.createDataFrame(example)
exampleDF.printSchema()

root
 |-- _1: string (nullable = true)
 |-- _2: long (nullable = true)
 |-- _3: string (nullable = true)
 |-- _4: string (nullable = true)



With columns names

In [ ]:
names = list(["name", "weight", "date", "country"])

In [ ]:
example2DF = example.toDF(names)
example2DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: long (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
# importing sql types
from pyspark.sql.types import *

In [ ]:
# custom schema
customSchema = StructType([ \
    StructField('name', StringType(), True), \
    StructField('weight', StringType(), True), \
    StructField('date', StringType(), True), \
    StructField('country', StringType(), True)])

In [ ]:
example3DF = spark.createDataFrame(example, customSchema)
example3DF.printSchema()

root
 |-- name: string (nullable = true)
 |-- weight: string (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
example3DF.show(2, False)

+-------------------------+------+----------+-------+
|name                     |weight|date      |country|
+-------------------------+------+----------+-------+
|chevrolet chevelle malibu|18    |1970-01-01|USA    |
|buick skylark 320        |15    |1970-01-01|USA    |
+-------------------------+------+----------+-------+



In [ ]:
# we can also specify schema with DDL (Data Definition Language)
customSchema2 = "`name` STRING NOT NULL, `weight` INT, `date` STRING, `country` STRING"

In [ ]:
example4DF = spark.createDataFrame(example, customSchema2)
example4DF.printSchema()

root
 |-- name: string (nullable = false)
 |-- weight: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- country: string (nullable = true)



In [ ]:
print(type(example2DF.collect()[0]["weight"]))
print(type(example3DF.collect()[0]["weight"]))

<class 'int'>
<class 'str'>


## Exercises
1) Create a manual DF describing smartphones
  - maker
  - model
  - screen dimension
  - camera megapixels
  
2) Read another file from the dataset/ folder, e.g. movies.json
  - print its schema
  - count the number of rows, call count()

3) Take a look to vehicles.csv. Read the file to a DF but this time with your own schema

Exercise 1

In [ ]:

smartphones = spark.sparkContext.parallelize([("iphone",4,"120x120","50mpx"),
    ("samsung",5,"50x60","60mpx")])



customSchema = StructType([ \
    StructField('maker', StringType(), True), \
    StructField('model', StringType(), True), \
    StructField('screen dimension', StringType(), True), \
    StructField('camera megapixels', StringType(), True)])



smartphones = spark.createDataFrame(smartphones,customSchema)
smartphones.printSchema()
smartphones.show(2, False)

Exercise 2

In [ ]:
moviesDF = spark.read.option("inferSchema", True).json("/dataset/movies.json")

moviesSchema = moviesDF.schema
print(type(moviesSchema))
print(moviesSchema)
moviesDF.printSchema()

moviesDF.count()

Exercise 3

In [ ]:

vehiclesDF = spark.read.option("header", "true").option("delimiter", ";").schema(customSchema).csv("/dataset/vehicles.csv")
vehiclesDF.show()